<a href="https://colab.research.google.com/github/srirampattabiraman/Extensive-NLP-and-Deep-Learning/blob/main/session_7/SSTB_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#We are mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
##Importing Necessary libraries afer thanking GOD

import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
from torchtext.legacy import data
from torchtext.legacy.data import Field, LabelField, BucketIterator
import random
from tqdm import tqdm
import sys
import os
from sklearn.model_selection import train_test_split

**Dataset Preparation**

In [4]:
def group_labels(label):
  if label in ["very negative", "negative"]:
    return "negative"
  elif label in ["positive", "very positive"]:
    return "positive" 
  else:
    return "neutral"

In [5]:
##Loading the Dataset
dictionary = pd.read_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/dictionary.txt", sep="|")
dictionary.columns = ["phrase", "id"]
dictionary = dictionary.set_index("id")

sentiment_labels = pd.read_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/sentiment_labels.txt", sep="|")
sentiment_labels.columns = ["id", "sentiment"]
sentiment_labels = sentiment_labels.set_index("id")

phrase_sentiments = dictionary.join(sentiment_labels)

phrase_sentiments["fine"] = pd.cut(phrase_sentiments.sentiment, [0, 0.2, 0.4, 0.6, 0.8, 1.0],
                                           include_lowest=True,
                                           labels=["very negative", "negative", "neutral", "positive", "very positive"])
phrase_sentiments["coarse"] = phrase_sentiments.fine.apply(group_labels)

phrase_sentiments = phrase_sentiments.reset_index(level=0)

sentences = pd.read_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/datasetSentences.txt", index_col="sentence_index",
                                sep="\t")

In [9]:
sentences.head()

,sentence
sentence_index,
1,The Rock is destined to be the 21st Century 's...
2,The gorgeously elaborate continuation of `` Th...
3,Effective but too-tepid biopic
4,If you sometimes like to go to the movies to h...
5,"Emerges as something rare , an issue movie tha..."


In [10]:
phrase_sentiments.head()

,id,phrase,sentiment,fine,coarse
0,22935,! ',0.52778,neutral,neutral
1,18235,! '',0.50000,neutral,neutral
2,179257,! Alas,0.44444,neutral,neutral
3,22936,! Brilliant,0.86111,very positive,positive
4,40532,! Brilliant !,0.93056,very positive,positive


In [6]:
##Merging Sentences, dictionary, sentiment labels
df_merged = pd.merge(sentences, phrase_sentiments, left_on='sentence', right_on='phrase')
display(df_merged.head())

,sentence,id,phrase,sentiment,fine,coarse
0,The Rock is destined to be the 21st Century 's...,226166,The Rock is destined to be the 21st Century 's...,0.69444,positive,positive
1,The gorgeously elaborate continuation of `` Th...,226300,The gorgeously elaborate continuation of `` Th...,0.83333,very positive,positive
2,Effective but too-tepid biopic,13995,Effective but too-tepid biopic,0.51389,neutral,neutral
3,If you sometimes like to go to the movies to h...,14123,If you sometimes like to go to the movies to h...,0.73611,positive,positive
4,"Emerges as something rare , an issue movie tha...",13999,"Emerges as something rare , an issue movie tha...",0.86111,very positive,positive


In [7]:
df_merged.to_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/sst_session_5.csv", index=False)

DATASET SPLIT

In [11]:
df = pd.read_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/sst_session_5.csv")
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42, stratify=df['coarse'])

In [12]:
df_train.to_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/sst_session_5_train.csv", index=False)
df_test.to_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/sst_session_5_test.csv", index=False)